<a href="https://colab.research.google.com/github/sarabert96/Colexification/blob/main/FastText_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install fasttext

In [2]:
import fasttext.util
from gensim.models.wrappers import FastText

In [3]:
from gensim.models.keyedvectors import KeyedVectors
from gensim.models import Word2Vec

Load pretrained model: wiki sample

In [4]:
model = KeyedVectors.load_word2vec_format('wiki.simple.vec')

Try to see the vector of few words:

In [5]:
# model['word'] --> access keyed vectors
model['house']

array([ 2.0695e-01,  7.9656e-02,  4.3752e-01,  1.1519e-01, -4.2894e-02,
        5.0550e-01, -1.9424e-02,  2.2890e-01,  2.2018e-01, -1.2672e-01,
       -1.1345e-01,  3.3551e-01,  4.6808e-01,  3.7312e-01, -2.5111e-01,
       -1.1842e-01,  1.6301e-01,  4.3348e-02, -4.7680e-01,  1.8059e-01,
        1.5040e-01,  1.5900e-01, -1.0340e-01, -5.9205e-02, -2.1054e-01,
       -1.8093e-01,  1.7380e-01,  3.3910e-02, -2.0083e-01,  3.3837e-01,
        4.6986e-01,  1.1231e-01, -3.9199e-03, -3.4872e-02,  2.2115e-01,
       -2.8744e-01, -3.9006e-01, -2.4090e-01, -1.6768e-01,  4.4849e-01,
        1.3874e-01, -4.9120e-02,  3.4582e-01,  1.5075e-01,  3.7149e-01,
        3.7103e-01, -1.4188e-01, -1.3651e-01,  8.6161e-02, -5.4532e-01,
        6.1135e-01, -7.0246e-03,  6.2059e-02,  2.5860e-02,  1.1958e-02,
       -1.2276e-01,  4.2305e-01, -1.3935e-01,  2.2550e-01,  2.2193e-01,
       -1.5392e-01,  3.7199e-01, -2.1911e-01,  6.4260e-04, -9.6433e-02,
       -3.8005e-01, -4.6484e-02, -1.9214e-01,  9.3353e-02,  1.16

In [6]:
model['HOUSE']

KeyError: ignored

Handle exception: word not in vocabulary

In [7]:
try:
  model.wv['HOUSE']
except:
  print ("word not in vocabulary")

word not in vocabulary


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


Try to calculate cosine similarity:

In [8]:
sim_score = model.similarity('house','home')
print(sim_score)

0.35669577


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Let's work with the dataframe

In [9]:
import numpy as np
import pandas as pd

Load the dataframe from csv

In [10]:
df = pd.read_csv('df_combinations.csv')

In [11]:
df.head()

,Concepticon_Gloss_1,Concepticon_Gloss_2,Concepticon_ID_1,Concepticon_ID_2
0,gold,good,1369,1035
1,gold,green,1369,1425
2,gold,hammer,1369,1481
3,gold,hand,1369,1277
4,gold,head,1369,1256


In [12]:
# inserted error to handle not in vocabulary exception
df['Concepticon_Gloss_1'].loc[0]='gooold'

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


Let's add the column with cosine similarity

In [13]:
# add new column with default 0
df['cosine_similarity']=0

# for every word in column1 and column2, calculate cosine similarity
for i in range(len(df['Concepticon_Gloss_1'])):
  try:
    w1 = df['Concepticon_Gloss_1'].loc[i]
    w2 = df['Concepticon_Gloss_2'].loc[i]
    cosine_similarity = model.similarity(w1,w2)
  except:
    # if the word is not in vocab, cosine similarity is 0
    cosine_similarity=0
  df['cosine_similarity'].loc[i]=cosine_similarity


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [14]:
# smarter way (not handling exception)
#add column with cosine sims row-wise
df['cosine_sim_2'] = df.apply(lambda row: model.similarity(row.Concepticon_Gloss_1, row.Concepticon_Gloss_2), axis=1) 

KeyError: ignored

In [15]:
# handle exception
def trySimilarity(w1, w2, default):
  try:
    return model.similarity(w1, w2)
  except:
    return default

df['cosine_sim_3'] = df.apply(lambda row: trySimilarity(row.Concepticon_Gloss_1, row.Concepticon_Gloss_2, 0), axis=1) 

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [16]:
# handle exception knowing word
def trySimilarity2(w1, w2, default):
  try:
    model[w1]
  except:
    print(w1)
    return 0
  try:
    model[w2]
  except:
    print(w2)
    return 0
  return model.similarity(w1, w2)

In [17]:
df['cosine_sim_4'] = df.apply(lambda row: trySimilarity2(row.Concepticon_Gloss_1, row.Concepticon_Gloss_2, 0), axis=1) 

gooold


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [18]:
df.head()

,Concepticon_Gloss_1,Concepticon_Gloss_2,Concepticon_ID_1,Concepticon_ID_2,cosine_similarity,cosine_sim_3,cosine_sim_4
0,gooold,good,1369,1035,0.000000,0.000000,0.000000
1,gold,green,1369,1425,0.185446,0.185446,0.185446
2,gold,hammer,1369,1481,0.228490,0.228490,0.228490
3,gold,hand,1369,1277,0.112822,0.112822,0.112822
4,gold,head,1369,1256,0.105320,0.105320,0.105320
